# 구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 드라이브 경로 지정
zip_folder = "/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음"

import glob, os
zip_files = glob.glob(os.path.join(zip_folder, "**", "*.zip"), recursive=True)
print(zip_files)

['/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202409.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202408.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202405.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202406.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202407.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202403.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202404.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202402.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202401.zip', '/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/카드소비 데이터_202412.zip', '/content

# 행정구역 코드 전처리

In [ ]:
import pandas as pd

# txt 파일 경로
file_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/법정동_행정구역 코드.txt"

# 파일 읽기
df = pd.read_csv(file_path, sep="|", encoding="cp949", dtype=str)

# 필요한 컬럼만 추출
df_selected = df[["ADM_CD", "ADM_SECT_NM"]].drop_duplicates()

# 10자리 코드에서 앞 8자리만 사용 → 시군구 단위
df_selected["ADM_CD"] = df_selected["ADM_CD"].str[:8]

# 뒷 두자리가 "00"인 경우만 필터링 (시군구 단위)
df_sgg = df_selected[df_selected["ADM_CD"].str.endswith("00")][["ADM_CD", "ADM_SECT_NM"]].drop_duplicates()

# 저장
output_path = "/content/drive/MyDrive/경기도_행정구역코드_시군구.csv"
df_sgg.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 저장 완료:", output_path)
print(df_sgg.head(20))

# 경기 카드소비 데이터 전처리

In [ ]:
import pandas as pd
import zipfile, glob, io, os

zip_folder = "/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음"

zip_files = glob.glob(os.path.join(zip_folder, "*.zip"))
df_list = []

for zf in zip_files:
    with zipfile.ZipFile(zf, 'r') as zip_ref:
        for fname in zip_ref.namelist():
            if not fname.endswith(".csv"):
                continue
            with zip_ref.open(fname) as f:
                try:
                    temp = pd.read_csv(f, encoding="utf-8")
                except:
                    try:
                        f.seek(0)
                        temp = pd.read_csv(f, encoding="cp949")
                    except:
                        f.seek(0)
                        temp = pd.read_csv(io.TextIOWrapper(f, encoding="latin1", errors="replace"))

                # 대분류/소분류 컬럼이 있을 때만 추출
                if {"card_tpbuz_nm_1","card_tpbuz_nm_2"}.issubset(temp.columns):
                    df_list.append(temp[["card_tpbuz_nm_1","card_tpbuz_nm_2"]])

# 하나로 합치고 중복 제거
category_map = (
    pd.concat(df_list, ignore_index=True)
      .drop_duplicates()
      .sort_values(["card_tpbuz_nm_1","card_tpbuz_nm_2"])
      .reset_index(drop=True)
)

# 저장
output_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/업종_대분류_소분류_전체매핑.csv"
category_map.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 전체 매핑 저장 완료:", output_path)
print(category_map.head(20))

In [ ]:
# category_map을 CSV로 저장
output_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/업종_대분류_소분류_매핑.csv"
category_map.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 저장 완료:", output_path)

In [ ]:
import pandas as pd
import zipfile, glob, os, io

# ======================
# 0. 경로 설정
# ======================
zip_folder = "/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음"
code_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/경기도_행정구역코드_시군구.csv"

# ======================
# 1) 생활형 / 비생활형 기준
# ======================
living_keywords = [
    "건강/기호식품","서적/도서","선물/완구","스포츠/레져용품","음/식료품소매",
    "의복/의류","인테리어/가정용품","종합소매점","패션잡화","기타용품",
    "가전제품","유아용품","화장품소매",
    "세탁/가사서비스","수리서비스","보안/운송","연료판매","사우나/휴게시설",
    "차량관리/서비스","차량관리/부품","음식배달서비스","교통서비스","미용서비스",
    "의약/의료품","일반병원","종합병원","특화병원","수의업","기타의료",
    "입시학원","유아교육","외국어학원","예체능계학원","독서실/고시원",
    "기타교육","기술/직업교육학원","자동차학원",
    "고기요리","닭/오리요리","별식/퓨전요리","부페","분식","양식",
    "일식/수산물","제과/제빵/떡/케익","중식","커피/음료","패스트푸드","한식"
]

nonliving_keywords = [
    "경기관람","공연관람","문화서비스","전시장","방송/미디어",
    "악기/공예","숙박","요가/단전/마사지","일반스포츠","취미/오락",
    "간이주점","유흥주점","여행/유학대행"
]

def classify_use(name: str) -> str:
    if not isinstance(name, str) or not name.strip():
        return "미분류"
    n = name.strip()
    if n in living_keywords:
        return "생활형"
    if n in nonliving_keywords:
        return "비생활형"
    return "기타"

# ======================
# 2. 행정구역 코드 매핑
# ======================
code_map = pd.read_csv(code_path, dtype=str)
code_map = code_map[["ADM_CD", "ADM_SECT_NM"]].drop_duplicates()
code_map.rename(columns={"ADM_CD": "admi_cty_no"}, inplace=True)

# ======================
# 3. ZIP 파일 탐색 & 처리
# ======================
zip_files = glob.glob(os.path.join(zip_folder, "*.zip"))
df_list = []

for zf in zip_files:
    base = os.path.basename(zf)
    year = int(base[-10:-6])
    month = int(base[-6:-4])
    quarter = (month - 1) // 3 + 1

    with zipfile.ZipFile(zf, 'r') as zip_ref:
        for fname in zip_ref.namelist():
            if not fname.endswith(".csv"):
                continue
            with zip_ref.open(fname) as f:
                try:
                    temp = pd.read_csv(f, encoding="utf-8")
                except:
                    try:
                        f.seek(0)
                        temp = pd.read_csv(f, encoding="cp949")
                    except:
                        f.seek(0)
                        temp = pd.read_csv(io.TextIOWrapper(f, encoding="latin1", errors="replace"))

                needed_cols = {"amt", "admi_cty_no", "card_tpbuz_nm_2"}
                if not needed_cols.issubset(temp.columns):
                    if "admi_dong_no" in temp.columns and "admi_cty_no" not in temp.columns:
                        temp = temp.rename(columns={"admi_dong_no": "admi_cty_no"})
                        if not needed_cols.issubset(temp.columns):
                            continue
                    else:
                        continue

                temp["card_tpbuz_nm_2"] = temp["card_tpbuz_nm_2"].astype(str).str.strip()
                temp["구분"] = temp["card_tpbuz_nm_2"].apply(classify_use)

                grouped = temp.groupby(["admi_cty_no", "구분"], as_index=False)["amt"].sum()
                grouped["연도"] = year
                grouped["분기"] = quarter
                df_list.append(grouped)

# ======================
# 4. 통합 및 매핑
# ======================
df_final = pd.concat(df_list, ignore_index=True)
df_final["admi_cty_no"] = df_final["admi_cty_no"].astype(str)
code_map["admi_cty_no"] = code_map["admi_cty_no"].astype(str)
df_final = df_final.merge(code_map, on="admi_cty_no", how="left")

# ======================
# 5. 컬럼 정리 & 분기 합산
# ======================
df_final = (
    df_final
    .groupby(["ADM_SECT_NM", "연도", "분기", "구분"], as_index=False)["amt"]
    .sum()
    .rename(columns={"ADM_SECT_NM": "지역", "amt": "소비금액"})
)

# ======================
# 6. 전체 합계 추가
# ======================
df_total = (
    df_final.groupby(["지역","연도","분기"], as_index=False)["소비금액"].sum()
)
df_total["구분"] = "전체"

# 합치기
df_final = pd.concat([df_final, df_total], ignore_index=True)

# ======================
# 7. 보기 좋은 단위
# ======================
df_final["소비금액(억)"] = (df_final["소비금액"] / 1e8).round(1)
df_final["소비금액(억표시)"] = df_final["소비금액(억)"].apply(lambda x: f"{x:,.1f} 억원")

# ======================
# 8. 정렬
# ======================
df_sorted = df_final.sort_values(
    by=["연도","분기","소비금액"],
    ascending=[True, True, False]
).reset_index(drop=True)

# ======================
# 9. 저장
# ======================
output_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시군구_분기별_생활비생활_전체합계포함.csv"
df_sorted.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 최종 데이터 생성 완료 및 정렬 완료:", output_path)
print(df_sorted.head(15))

✅ 최종 데이터 생성 완료 및 정렬 완료: /content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시군구_분기별_생활비생활_전체합계포함.csv
                  지역    연도  분기   구분          소비금액  소비금액(억)   소비금액(억표시)
0   경기도 수원시 영통구 광교1동  2024   1   전체  159391467964   1593.9  1,593.9 억원
1   경기도 수원시 영통구 광교1동  2024   1  생활형  148284566769   1482.8  1,482.8 억원
2       경기도 화성시 동탄6동  2024   1   전체  128045029274   1280.5  1,280.5 억원
3       경기도 화성시 동탄6동  2024   1  생활형  119892430235   1198.9  1,198.9 억원
4   경기도 안산시 단원구 선부3동  2024   1   전체   96587649854    965.9    965.9 억원
5   경기도 안산시 단원구 선부3동  2024   1  생활형   89956716514    899.6    899.6 억원
6     경기도 수원시 팔달구 지동  2024   1   전체   54109669250    541.1    541.1 억원
7     경기도 수원시 팔달구 지동  2024   1  생활형   52431339310    524.3    524.3 억원
8    경기도 안산시 상록구 반월동  2024   1   전체   38221367646    382.2    382.2 억원
9    경기도 안산시 상록구 반월동  2024   1  생활형   36719286087    367.2    367.2 억원
10       경기도 하남시 초이동  2024   1   전체   26749120747    267.5    267.5 억원
11  경기도 안양시 동안구 호계3동  2024   1   전체   25217937578

In [ ]:
# 구분이 '기타'인 데이터만 추출
df_etc = df_final[df_final["구분"] == "기타"]

# 어떤 업종(card_tpbuz_nm_2)이 '기타'로 분류되었는지 보고 싶다면
# (원본 temp에서 바로 확인 필요)
etc_list = temp[temp["card_tpbuz_nm_2"].apply(classify_use) == "기타"]["card_tpbuz_nm_2"].unique()
print("기타 분류 업종:", etc_list)

기타 분류 업종: ['사무/교육용품' '차량판매' '제조/도매' '광고/인쇄/인화' '부동산' '무점포서비스' '회비/공과금' '휴게소/대형업체'
 '인터넷쇼핑' '기타결제' '학교' '기업' '단체' '방문판매' '전문서비스' '가례서비스' '금융상품/서비스' '렌탈서비스'
 '시스템/통신' '공공기관' '종교']


In [ ]:
import re
import pandas as pd

# df_sorted 가 이미 만들어져 있다고 가정 (지역, 연도, 분기, 구분, 소비금액 …)

# ---------------------------------------
# 1) 지역 문자열에서 "시만" 또는 "시+구"를 추출하는 함수
# ---------------------------------------
def extract_match_region(name: str) -> str:
    """
    예시
    - '경기도 수원시 영통구 광교1동' -> '경기도 수원시 영통구'
    - '경기도 과천시 별양동'         -> '경기도 과천시'
    - '경기도 양평군 양평읍'         -> '경기도 양평군'
    """
    if not isinstance(name, str):
        return name
    toks = name.split()

    # 시/군/구 토큰 위치 찾기
    si_idx = None
    gu_idx = None

    for i, t in enumerate(toks):
        if t.endswith("시") or t.endswith("군"):
            si_idx = i
            # 이후에 '구'가 있으면 그 위치 기억
            for j in range(i+1, len(toks)):
                if toks[j].endswith("구"):
                    gu_idx = j
                    break
            break
        # (간혹 바로 '구'부터 나오는 데이터 방어)
        if t.endswith("구") and gu_idx is None:
            gu_idx = i

    # 우선순위: 시 찾음 → 그 뒤 구 있으면 시~구, 없으면 시까지
    if si_idx is not None:
        if gu_idx is not None and gu_idx > si_idx:
            return " ".join(toks[:gu_idx+1])  # … 시 … 구
        else:
            return " ".join(toks[:si_idx+1])  # … 시
    # 시/군이 없고 구만 있다면 구까지
    if gu_idx is not None:
        return " ".join(toks[:gu_idx+1])

    # 규칙 매칭 실패 시 원본 반환
    return name

# ---------------------------------------
# 2) 지역_매칭 만들고 해당 단위로 재집계
# ---------------------------------------
df_matched = df_sorted.copy()
df_matched["지역_매칭"] = df_matched["지역"].apply(extract_match_region)

# (전체는 이미 포함되어 있으므로 그대로 유지해도 되지만,
#  혹시를 위해 '전체'는 다시 계산하는 버전으로 만드는 게 안전하다면 아래처럼 재계산하세요.)
# 먼저 생활/비생활/기타만 필터
df_base = df_matched[df_matched["구분"].isin(["생활형", "비생활형", "기타"])]

# 지역_매칭 × 연도 × 분기 × 구분 합계
df_grp = (
    df_base.groupby(["지역_매칭", "연도", "분기", "구분"], as_index=False)["소비금액"]
          .sum()
)

# 전체 합계(생활+비생활+기타)
df_total = (
    df_grp.groupby(["지역_매칭", "연도", "분기"], as_index=False)["소비금액"]
          .sum()
)
df_total["구분"] = "전체"

# 합치기
df_out = pd.concat([df_grp, df_total], ignore_index=True)

# 보기 좋은 단위
df_out["소비금액(억)"] = (df_out["소비금액"] / 1e8).round(1)
df_out["소비금액(억표시)"] = df_out["소비금액(억)"].apply(lambda x: f"{x:,.1f} 억원")

# 정렬: 연도, 분기 오름차순 / 소비금액 내림차순
df_out = df_out.sort_values(
    by=["연도", "분기", "소비금액"],
    ascending=[True, True, False]
).reset_index(drop=True)

# 저장
save_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시군구기준_생활비생활_전체합계포함.csv"
df_out.to_csv(save_path, index=False, encoding="utf-8-sig")
print("✅ 시/구 기준으로 재집계 완료:", save_path)
print(df_out.head(10))

✅ 시/구 기준으로 재집계 완료: /content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시군구기준_생활비생활_전체합계포함.csv
         지역_매칭    연도  분기   구분          소비금액  소비금액(억)   소비금액(억표시)
0      경기도 화성시  2024   1   전체  267037757226   2670.4  2,670.4 억원
1      경기도 화성시  2024   1  생활형  200397707035   2004.0  2,004.0 억원
2  경기도 수원시 영통구  2024   1   전체  159391467964   1593.9  1,593.9 억원
3  경기도 수원시 영통구  2024   1  생활형  148284566769   1482.8  1,482.8 억원
4  경기도 안산시 단원구  2024   1   전체   96587649854    965.9    965.9 억원
5  경기도 안산시 단원구  2024   1  생활형   89956716514    899.6    899.6 억원
6      경기도 화성시  2024   1   기타   60229711148    602.3    602.3 억원
7  경기도 수원시 팔달구  2024   1   전체   54109669250    541.1    541.1 억원
8  경기도 수원시 팔달구  2024   1  생활형   52431339310    524.3    524.3 억원
9  경기도 안산시 상록구  2024   1   전체   38221367646    382.2    382.2 억원


In [4]:
import re
import pandas as pd

# df_sorted 가 기존 카드소비 데이터 (지역, 연도, 분기, 구분, 소비금액 …)
df_sorted= pd.read_csv('/content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시군구_분기별_생활비생활_전체합계포함.csv')
# ---------------------------------------
# 1) "시/군"까지만 추출하는 함수
# ---------------------------------------
def extract_city(name: str) -> str:
    """
    예시
    - '경기도 수원시 영통구 광교1동' -> '경기도 수원시'
    - '경기도 과천시 별양동'         -> '경기도 과천시'
    - '경기도 양평군 양평읍'         -> '경기도 양평군'
    """
    if not isinstance(name, str):
        return name
    toks = name.split()

    for i, t in enumerate(toks):
        if t.endswith("시") or t.endswith("군"):   # 시 / 군 찾으면 거기까지 반환
            return " ".join(toks[:i+1])
    return name

# ---------------------------------------
# 2) 지역_시 기준으로 집계
# ---------------------------------------
df_city = df_sorted.copy()
df_city["지역_시"] = df_city["지역"].apply(extract_city)

# 우선 생활형/비생활형/기타만 합산
df_base = df_city[df_city["구분"].isin(["생활형","비생활형","기타"])]

df_grp = (
    df_base.groupby(["지역_시","연도","분기","구분"], as_index=False)["소비금액"]
           .sum()
)

# 전체 추가
df_total = (
    df_grp.groupby(["지역_시","연도","분기"], as_index=False)["소비금액"]
          .sum()
)
df_total["구분"] = "전체"

df_out = pd.concat([df_grp, df_total], ignore_index=True)

# 보기 좋은 단위
df_out["소비금액(억)"] = (df_out["소비금액"] / 1e8).round(1)
df_out["소비금액(억표시)"] = df_out["소비금액(억)"].apply(lambda x: f"{x:,.1f} 억원")

# 정렬
df_out = df_out.sort_values(
    by=["연도","분기","소비금액"],
    ascending=[True,True,False]
).reset_index(drop=True)

# 저장
save_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시기준_생활비생활_전체합계포함.csv"
df_out.to_csv(save_path, index=False, encoding="utf-8-sig")

print("✅ 시 단위 집계 완료:", save_path)
print(df_out.head(10))

✅ 시 단위 집계 완료: /content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시기준_생활비생활_전체합계포함.csv
      지역_시    연도  분기   구분          소비금액  소비금액(억)   소비금액(억표시)
0  경기도 화성시  2024   1   전체  267037757226   2670.4  2,670.4 억원
1  경기도 수원시  2024   1   전체  244732600664   2447.3  2,447.3 억원
2  경기도 수원시  2024   1  생활형  229685610348   2296.9  2,296.9 억원
3  경기도 화성시  2024   1  생활형  200397707035   2004.0  2,004.0 억원
4  경기도 안산시  2024   1   전체  134809017500   1348.1  1,348.1 억원
5  경기도 안산시  2024   1  생활형  126676002601   1266.8  1,266.8 억원
6  경기도 화성시  2024   1   기타   60229711148    602.3    602.3 억원
7  경기도 안양시  2024   1   전체   49236567811    492.4    492.4 억원
8  경기도 안양시  2024   1  생활형   44292990710    442.9    442.9 억원
9  경기도 하남시  2024   1   전체   26749120747    267.5    267.5 억원


# 아파트 매매 지수/전세 지수 전처리

In [ ]:
import pandas as pd

# 파일 읽기
df = pd.read_excel("/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/(월) 지역별 매매지수_아파트.xlsx")

# 2024년 컬럼만 선택
cols_2024 = ["2024년 1월","2024년 2월","2024년 3월",
             "2024년 4월","2024년 5월","2024년 6월",
             "2024년 7월","2024년 8월","2024년 9월",
             "2024년 10월","2024년 11월","2024년 12월"]

df_2024 = df.loc[:, ["지역"] + cols_2024].copy()

# 모든 월 컬럼 숫자로 변환 (문자 → NaN)
for col in cols_2024:
    df_2024[col] = pd.to_numeric(df_2024[col], errors="coerce")

# 분기별 평균 계산
df_2024["2024_1분기"] = df_2024[["2024년 1월","2024년 2월","2024년 3월"]].mean(axis=1, skipna=True)
df_2024["2024_2분기"] = df_2024[["2024년 4월","2024년 5월","2024년 6월"]].mean(axis=1, skipna=True)
df_2024["2024_3분기"] = df_2024[["2024년 7월","2024년 8월","2024년 9월"]].mean(axis=1, skipna=True)
df_2024["2024_4분기"] = df_2024[["2024년 10월","2024년 11월","2024년 12월"]].mean(axis=1, skipna=True)

# 최종 테이블
df_final = df_2024[["지역","2024_1분기","2024_2분기","2024_3분기","2024_4분기"]]

# 저장
df_final.to_csv("/content/drive/MyDrive/my_ws/SeSAC_Python/매매지수_2024분기.csv", index=False, encoding="utf-8-sig")

print(df_final.head())

    지역    2024_1분기    2024_2분기    2024_3분기    2024_4분기
0   지역         NaN         NaN         NaN         NaN
1   전국  122.800000  123.000000  125.000000  124.733333
2  수도권  141.500000  143.033333  147.100000  147.100000
3   지방  107.233333  106.433333  106.600000  106.166667
4   서울  159.333333  162.500000  170.033333  171.066667


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
import pandas as pd

# 파일 읽기
df = pd.read_excel("/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음/(월) 지역별 전세지수_아파트.xlsx")

# 2024년 컬럼만 선택
cols_2024 = ["2024년 1월","2024년 2월","2024년 3월",
             "2024년 4월","2024년 5월","2024년 6월",
             "2024년 7월","2024년 8월","2024년 9월",
             "2024년 10월","2024년 11월","2024년 12월"]

df_2024 = df.loc[:, ["지역"] + cols_2024].copy()

# 모든 월 컬럼 숫자로 변환 (문자 → NaN)
for col in cols_2024:
    df_2024[col] = pd.to_numeric(df_2024[col], errors="coerce")

# 분기별 평균 계산
df_2024["2024_1분기"] = df_2024[["2024년 1월","2024년 2월","2024년 3월"]].mean(axis=1, skipna=True)
df_2024["2024_2분기"] = df_2024[["2024년 4월","2024년 5월","2024년 6월"]].mean(axis=1, skipna=True)
df_2024["2024_3분기"] = df_2024[["2024년 7월","2024년 8월","2024년 9월"]].mean(axis=1, skipna=True)
df_2024["2024_4분기"] = df_2024[["2024년 10월","2024년 11월","2024년 12월"]].mean(axis=1, skipna=True)

# 최종 테이블
df_final = df_2024[["지역","2024_1분기","2024_2분기","2024_3분기","2024_4분기"]]

# 저장
df_final.to_csv("/content/drive/MyDrive/my_ws/SeSAC_Python/전세지수_2024분기.csv", index=False, encoding="utf-8-sig")

print(df_final.head())

    지역    2024_1분기    2024_2분기    2024_3분기    2024_4분기
0   지역         NaN         NaN         NaN         NaN
1   지역         NaN         NaN         NaN         NaN
2   전국  120.733333  122.166667  124.133333  123.666667
3  수도권  123.766667  125.966667  128.366667  127.466667
4   지방  114.733333  114.300000  115.266667  115.833333


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
import pandas as pd
import zipfile, glob, os, io

# ======================
# 0. 경로 설정
# ======================
zip_folder = "/content/drive/MyDrive/my_ws/SeSAC_Python/파이썬 프젝 데이터 모음"
code_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/경기도_행정구역코드_시군구.csv"

# ======================
# 1) 생활형 / 비생활형 기준
# ======================
living_keywords = [
    "건강/기호식품","서적/도서","선물/완구","스포츠/레져용품","음/식료품소매",
    "의복/의류","인테리어/가정용품","종합소매점","패션잡화","기타용품",
    "가전제품","유아용품","화장품소매",
    "세탁/가사서비스","수리서비스","보안/운송","연료판매","사우나/휴게시설",
    "차량관리/서비스","차량관리/부품","음식배달서비스","교통서비스","미용서비스",
    "의약/의료품","일반병원","종합병원","특화병원","수의업","기타의료",
    "입시학원","유아교육","외국어학원","예체능계학원","독서실/고시원",
    "기타교육","기술/직업교육학원","자동차학원",
    "고기요리","닭/오리요리","별식/퓨전요리","부페","분식","양식",
    "일식/수산물","제과/제빵/떡/케익","중식","커피/음료","패스트푸드","한식"
]

nonliving_keywords = [
    "경기관람","공연관람","문화서비스","전시장","방송/미디어",
    "악기/공예","숙박","요가/단전/마사지","일반스포츠","취미/오락",
    "간이주점","유흥주점","여행/유학대행"
]

def classify_use(name: str) -> str:
    """업종명을 생활형/비생활형/기타로 분류"""
    if not isinstance(name, str) or not name.strip():
        return "미분류"
    n = name.strip()
    if n in living_keywords:
        return "생활형"
    if n in nonliving_keywords:
        return "비생활형"
    return "기타"

# ======================
# 2. 행정구역 코드 매핑 (시군구 단위)
# ======================
code_map = pd.read_csv(code_path, dtype=str)
code_map = code_map.rename(columns={"ADM_CD": "admi_cty_no", "ADM_SECT_NM": "지역명"})
code_map["admi_cty_no"] = code_map["admi_cty_no"].str[:5]  # 시군구 단위 코드만 사용

# ======================
# 3. ZIP 파일 탐색 & 처리
# ======================
zip_files = glob.glob(os.path.join(zip_folder, "*.zip"))
df_list = []

for zf in zip_files:
    base = os.path.basename(zf)
    year = int(base[-10:-6])
    month = int(base[-6:-4])
    quarter = (month - 1) // 3 + 1

    with zipfile.ZipFile(zf, 'r') as zip_ref:
        for fname in zip_ref.namelist():
            if not fname.endswith(".csv"):
                continue
            with zip_ref.open(fname) as f:
                # 인코딩 시도
                try:
                    temp = pd.read_csv(f, encoding="utf-8")
                except:
                    try:
                        f.seek(0)
                        temp = pd.read_csv(f, encoding="cp949")
                    except:
                        f.seek(0)
                        temp = pd.read_csv(io.TextIOWrapper(f, encoding="latin1", errors="replace"))

                # 필요한 컬럼 확인
                needed_cols = {"amt", "admi_cty_no", "card_tpbuz_nm_2"}
                if not needed_cols.issubset(temp.columns):
                    if "admi_dong_no" in temp.columns and "admi_cty_no" not in temp.columns:
                        temp = temp.rename(columns={"admi_dong_no": "admi_cty_no"})
                        if not needed_cols.issubset(temp.columns):
                            continue
                    else:
                        continue

                # 동단위 코드 → 시군구 코드로 변환
                temp["admi_cty_no"] = temp["admi_cty_no"].astype(str).str[:5]

                # 업종 분류
                temp["card_tpbuz_nm_2"] = temp["card_tpbuz_nm_2"].astype(str).str.strip()
                temp["구분"] = temp["card_tpbuz_nm_2"].apply(classify_use)

                grouped = temp.groupby(["admi_cty_no", "구분"], as_index=False)["amt"].sum()
                grouped["연도"] = year
                grouped["분기"] = quarter
                df_list.append(grouped)

# ======================
# 4. 통합 및 매핑
# ======================
df_final = pd.concat(df_list, ignore_index=True)
df_final["admi_cty_no"] = df_final["admi_cty_no"].astype(str)

# 시군구 단위 매핑
df_final = df_final.merge(code_map, on="admi_cty_no", how="left")

# 매핑 안 된 지역 확인
missing = df_final[df_final["지역명"].isna()]
print("❌ 매핑 안 된 행 개수:", len(missing))
if len(missing) > 0:
    print(missing["admi_cty_no"].unique()[:20])

# ======================
# 5. 컬럼 정리 & 분기 합산
# ======================
df_final = (
    df_final
    .groupby(["지역명", "연도", "분기", "구분"], as_index=False)["amt"]
    .sum()
    .rename(columns={"amt": "소비금액", "지역명": "지역"})
)

# 전체 합계 추가
df_total = df_final.groupby(["지역","연도","분기"], as_index=False)["소비금액"].sum()
df_total["구분"] = "전체"
df_final = pd.concat([df_final, df_total], ignore_index=True)

# 보기 좋은 단위
df_final["소비금액(억)"] = (df_final["소비금액"] / 1e8).round(1)
df_final["소비금액(억표시)"] = df_final["소비금액(억)"].apply(lambda x: f"{x:,.1f} 억원")

# 정렬
df_sorted = df_final.sort_values(
    by=["연도","분기","소비금액"],
    ascending=[True, True, False]
).reset_index(drop=True)

# 저장
output_path = "/content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시군구_분기별.csv"
df_sorted.to_csv(output_path, index=False, encoding="utf-8-sig")

print("✅ 최종 데이터 생성 완료:", output_path)
print(df_sorted.head(15))

❌ 매핑 안 된 행 개수: 0
✅ 최종 데이터 생성 완료: /content/drive/MyDrive/my_ws/SeSAC_Python/경기_카드소비_시군구_분기별.csv
                 지역    연도  분기   구분           소비금액  소비금액(억)    소비금액(억표시)
0           경기도 화성시  2024   1   전체  2323618245912  23236.2  23,236.2 억원
1      경기도 화성시 동탄6동  2024   1   전체  2323618245912  23236.2  23,236.2 억원
2       경기도 화성시 양감면  2024   1   전체  2323618245912  23236.2  23,236.2 억원
3   경기도 화성시 양감면 대양리  2024   1   전체  2323618245912  23236.2  23,236.2 억원
4   경기도 화성시 양감면 사창리  2024   1   전체  2323618245912  23236.2  23,236.2 억원
5   경기도 화성시 양감면 송산리  2024   1   전체  2323618245912  23236.2  23,236.2 억원
6   경기도 화성시 양감면 신왕리  2024   1   전체  2323618245912  23236.2  23,236.2 억원
7   경기도 화성시 양감면 요당리  2024   1   전체  2323618245912  23236.2  23,236.2 억원
8   경기도 화성시 양감면 용소리  2024   1   전체  2323618245912  23236.2  23,236.2 억원
9   경기도 화성시 양감면 정문리  2024   1   전체  2323618245912  23236.2  23,236.2 억원
10          경기도 화성시  2024   1  생활형  1733672094477  17336.7  17,336.7 억원
11     경기도 화성시 동탄6동  2024   1  생활형  17336